# Predicting Human Ancestry Using SNP Data

This notebook demonstrates ancestry prediction using **ancestry-informative SNPs (AISNPs)** from the 1000 Genomes Project. We explore multiple models—**K-Nearest Neighbors (KNN), Random Forests, and Neural Networks**—and evaluate them with a consistent train/validation/test split.


## 1. Dataset Overview

The dataset was obtained from kaggle: https://www.kaggle.com/competitions/1000-genomes-ancestry

- **Samples:** 1,878 individuals  
- **Features:** 169 SNPs (encoded as 2-letter alleles)  
- **Target:** `superpopulation` (AFR, AMR, EAS, EUR, SAS)  

**Goal:** Predict the probability of each individual belonging to one of the five superpopulations.


### Package imports

In [306]:
import numpy as np
import pandas as pd
import itertools
import copy
from collections import Counter
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import torch
import torch.nn as nn
import torch.optim as optim

# Set seeds for reproducibility
seed=1
np.random.seed(seed)
torch.manual_seed(seed)

## 2. Preprocessing
### Load data

In [307]:
# Load
kidd_train = pd.read_csv('1000-genomes-ancestry/kidd_train.csv')
seldin_train = pd.read_csv('1000-genomes-ancestry/seldin_train.csv')
# Merge sets
kidd_variants = [x for x in kidd_train.columns if x.startswith('rs')]
seldin_variants = [x for x in seldin_train.columns if x.startswith('rs') and not x in kidd_variants]
data = pd.concat([kidd_train.loc[:,kidd_variants], seldin_train.loc[:,seldin_variants], seldin_train.loc[:,'superpopulation']], axis=1)
# Split X and y
X = data.drop('superpopulation', axis=1)
y = data['superpopulation']

### 2.1 Missing Genotype Imputation

Missing SNPs are filled with the **most frequent genotype (mode)** in the training set. Using the most frequent genotype in the training data prevents any data leakage between training and test sets. 

In [308]:
def impute_missing_genotypes(train_df: pd.DataFrame, test_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Impute missing genotypes in train and test sets with the mode of the training data.

    Parameters
    ----------
    train_df : pd.DataFrame
        Training features (only SNP columns, no id/label).
    test_df : pd.DataFrame
        Test features (only SNP columns, no id).

    Returns
    -------
    train_imputed : pd.DataFrame
        Training data with missing values filled.
    test_imputed : pd.DataFrame
        Test data with missing values filled.
    """
    # Compute mode per column from training
    modes = train_df.mode().iloc[0]

    # Fill missing in train and test
    train_imputed = train_df.fillna(modes)
    test_imputed = test_df.fillna(modes)

    return train_imputed, test_imputed

### 2.2 Numeric Encoding of Genotypes

Each SNP is converted to numeric representation relative to the **major allele** in the training set:

- `0` → homozygous major allele  
- `1` → heterozygous  
- `2` → homozygous minor allele  

Using the training set only to set the reference allele once again prevents data leakage.

In [309]:
def encode_genotypes(train_df: pd.DataFrame, val_df: pd.DataFrame):
    """
    Encode SNP genotypes as 0/1/2 using the major allele in training set.
    Avoids DataFrame fragmentation by building columns first and concatenating.
    
    Parameters
    ----------
    train_df : pd.DataFrame
        Training SNP genotypes (2-letter strings).
    val_df : pd.DataFrame
        Validation SNP genotypes.
    
    Returns
    -------
    train_enc : pd.DataFrame
        Encoded training SNPs.
    val_enc : pd.DataFrame
        Encoded validation SNPs.
    ref_dict : dict
        SNP -> major allele mapping.
    """
    ref_dict = {}
    train_cols, val_cols = {}, {}

    for snp in train_df.columns:
        # Determine major allele in training set
        alleles = []
        for g in train_df[snp].dropna():
            alleles.extend(list(g))
        major = Counter(alleles).most_common(1)[0][0]
        ref_dict[snp] = major

        def encode(g):
            if pd.isna(g):
                return np.nan
            a1, a2 = g[0], g[1]
            if a1 == major and a2 == major:
                return 0
            elif a1 == major or a2 == major:
                return 1
            else:
                return 2

        train_cols[snp] = train_df[snp].map(encode)
        val_cols[snp] = val_df[snp].map(encode)

    # Concatenate all columns at once
    train_enc = pd.concat(train_cols.values(), axis=1)
    train_enc.columns = train_cols.keys()
    train_enc = train_enc.copy()  # de-fragment

    val_enc = pd.concat(val_cols.values(), axis=1)
    val_enc.columns = val_cols.keys()
    val_enc = val_enc.copy()  # de-fragment

    return train_enc, val_enc, ref_dict

### 2.3 Feature Scaling

Features are standardized using `StandardScaler` to improve convergence for neural networks and distance-based models.

## 3. Train / Validation / Test Split

- **Training set:** for fitting models  
- **Validation set:** for hyperparameter tuning and early stopping  
- **Test set:** for final unbiased evaluation  

**Note:** The same validation set is used across models for fair comparison.


In [310]:
# First split: train+val vs test
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=seed
)

# Second split: train vs val
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval,
    test_size=0.2,   # 20% of train+val → ~16% of full dataset
    stratify=y_trainval,
    random_state=seed
)

## 4. K-Nearest Neighbors (KNN)

- Input: preprocessed SNPs (imputed and numerically encoded)  
- Hyperparameter tuning: number of neighbors `K`, distance weights  
- Metrics: **log loss** and **accuracy** on validation/test set


In [311]:
# Impute missing genotypes by mode of training set
X_train_imputed, X_val_imputed = impute_missing_genotypes(X_train, X_val)
X_train_imputed, X_val_imputed = impute_missing_genotypes(X_train, X_val)

In [312]:
# Encode genotypes numerically
X_train_enc, X_val_enc, ref_dict = encode_genotypes(X_train_imputed, X_val_imputed)

In [313]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_enc)
X_val_scaled = scaler.transform(X_val_enc)

In [314]:
# Encode labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)

In [315]:
# KNN model

# Define hyperparameter grid
param_grid = {
    "n_neighbors": [5, 10, 15, 20],
    "weights": ["uniform", "distance"]
}

# Use validation set via GridSearchCV with train/val split
knn = KNeighborsClassifier(n_jobs=-1)
grid = GridSearchCV(knn, param_grid, scoring='neg_log_loss', cv=[(np.arange(len(X_train_scaled)), np.arange(len(X_val_scaled)) + len(X_train_scaled))])
grid.fit(np.vstack([X_train_scaled, X_val_scaled]), np.hstack([y_train_enc, y_val_enc]))

# Retrieve the best model parameters
best_knn = grid.best_estimator_

In [316]:
# Evaluate on test set

# Prepare test set
_, X_test_imputed = impute_missing_genotypes(X_train, X_test)
_, X_test_enc, _ = encode_genotypes(X_train, X_test_imputed)
X_test_scaled = scaler.transform(X_test_enc)
y_test_enc = le.transform(y_test)

# Predict based on test set
y_test_proba = best_knn.predict_proba(X_test_scaled)
y_test_pred = best_knn.predict(X_test_scaled)

# Compute logloss and accuracy
test_logloss = log_loss(y_test_enc, y_test_proba)
test_acc = accuracy_score(y_test_enc, y_test_pred)

print(f"Test Log Loss: {test_logloss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

test_logloss_knn = test_logloss
test_acc_knn = test_acc

Test Log Loss: 0.7369
Test Accuracy: 0.9202


## 5. Random Forest

- Input: preprocessed SNPs  
- Hyperparameter tuning: `n_estimators`, `max_depth`, `max_features`  
- Validation set selects best model; retrained on **train + validation**  
- Class imbalance handled with `class_weight='balanced'`  
- Metrics: log loss and accuracy on test set

In [317]:
# Impute missing genotypes
X_train_imputed, X_val_imputed = impute_missing_genotypes(X_train, X_val)

In [318]:
# Encode genotypes numerically
X_train_enc, X_val_enc, ref_dict = encode_genotypes(X_train_imputed, X_val_imputed)

In [319]:
# Random Forest works fine without scaling
X_train_scaled = X_train_enc.values
X_val_scaled = X_val_enc.values

In [320]:
# Encode labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)

In [321]:
# Define hyperparameter grid
param_grid = {
    "n_estimators": [100, 300, 500],
    "max_depth": [None, 10, 20],
    "max_features": ["sqrt", "log2", 0.5]
}

# Prepare to store results
results = []

# Iterate over all combinations
for n_est, max_d, max_feat in itertools.product(
    param_grid["n_estimators"],
    param_grid["max_depth"],
    param_grid["max_features"]
):
    rf = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=max_d,
        max_features=max_feat,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_train_scaled, y_train_enc)
    
    y_val_proba = rf.predict_proba(X_val_scaled)
    y_val_pred = rf.predict(X_val_scaled)
    
    val_loss = log_loss(y_val_enc, y_val_proba)
    val_acc = accuracy_score(y_val_enc, y_val_pred)
    
    results.append({
        "n_estimators": n_est,
        "max_depth": max_d,
        "max_features": max_feat,
        "val_logloss": val_loss,
        "val_accuracy": val_acc
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df["max_depth"] = results_df["max_depth"].apply(lambda x: None if pd.isna(x) else int(x))

In [322]:
# Find best hyperparameters (lowest validation log loss)
best_row = results_df.loc[results_df["val_logloss"].idxmin()]
best_params = {
    "n_estimators": best_row["n_estimators"],
    "max_depth": best_row["max_depth"],
    "max_features": best_row["max_features"],
    "class_weight": "balanced",
    "random_state": seed,
    "n_jobs": -1
}
if pd.isna(best_params['max_depth']): best_params['max_depth'] = None
print("Best hyperparameters:", best_params)

Best hyperparameters: {'n_estimators': np.int64(500), 'max_depth': None, 'max_features': 0.5, 'class_weight': 'balanced', 'random_state': 1, 'n_jobs': -1}


In [323]:
# Train random forest on the cont=catenation of the training and validation sets
# Combine train + validation sets
X_trainval_combined = np.vstack([X_train_scaled, X_val_scaled])
y_trainval_combined = np.hstack([y_train_enc, y_val_enc])

# Retrain Random Forest on full training data
rf_best = RandomForestClassifier(**best_params)
rf_best.fit(X_trainval_combined, y_trainval_combined)

,n_estimators,np.int64(500)
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,0.5
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [324]:
# Evaluate on test set
# Impute missing genotypes and encode numerically for test set
# Prepare test set
_, X_test_imputed = impute_missing_genotypes(X_train, X_test)
_, X_test_enc, _ = encode_genotypes(X_train, X_test_imputed)
X_test_scaled = X_test_enc.values
y_test_enc = le.transform(y_test)

y_test_proba = rf_best.predict_proba(X_test_scaled)
y_test_pred = rf_best.predict(X_test_scaled)

test_logloss = log_loss(y_test_enc, y_test_proba)
test_acc = accuracy_score(y_test_enc, y_test_pred)

print(f"Test Log Loss: {test_logloss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

test_logloss_rf = test_logloss
test_acc_rf = test_acc

Test Log Loss: 0.1493
Test Accuracy: 0.9601


## 6. Neural Network (Single)

- Architecture: 2 hidden layers with ReLU and Dropout  
- **Elastic Net regularization** (L1 + L2)  
- **Early stopping** monitors validation loss and saves the best model  
- Input: preprocessed SNPs  
- Metrics: log loss and accuracy on test set

In [325]:
# Impute missing genotypes
X_train_imputed, X_val_imputed = impute_missing_genotypes(X_train, X_val)

In [326]:
# Encode genotypes numerically (0/1/2) using major allele
X_train_enc, X_val_enc, ref_dict = encode_genotypes(X_train_imputed, X_val_imputed)

In [327]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_enc)
X_val_scaled = scaler.transform(X_val_enc)

In [328]:
# Encode labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)

In [329]:
# Convert to torch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_enc, dtype=torch.long)
y_val_tensor = torch.tensor(y_val_enc, dtype=torch.long)

In [330]:
# Define neural network
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, output_dim)
        )

    def forward(self, x):
        return self.net(x)

input_dim = X_train_scaled.shape[1]
hidden_dim = 128
output_dim = len(le.classes_)
model = SimpleNN(input_dim, hidden_dim, output_dim)

In [331]:
# Optimizer with Elastic Net (L1 + L2)
l1_lambda = 1e-5
l2_lambda = 1e-4
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=l2_lambda)
criterion = nn.CrossEntropyLoss()

In [332]:
# Early stopping
class EarlyStopping:
    def __init__(self, patience=10, min_delta=1e-4):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = np.inf
        self.counter = 0
        self.stop = False
        self.best_model_state = None

    def step(self, val_loss, model):
        if val_loss + self.min_delta < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
            # Save a copy of the model state dict
            self.best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.stop = True

early_stopper = EarlyStopping(patience=10)

In [333]:
# Training loop
epochs = 100
batch_size = 64
dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        # L1 regularization
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss += l1_lambda * l1_norm
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_out = model(X_val_tensor)
        val_probs = torch.softmax(val_out, dim=1).cpu().numpy()
        val_loss = log_loss(y_val_enc, val_probs)
        val_acc = accuracy_score(y_val_enc, np.argmax(val_probs, axis=1))

    print(f"Epoch {epoch+1}: val_logloss={val_loss:.4f}, val_acc={val_acc:.4f}")
    early_stopper.step(val_loss, model)
    if early_stopper.stop:
        print(f"Early stopping at epoch {epoch+1}")
        break

# Load the best model before test evaluation
model.load_state_dict(early_stopper.best_model_state)
model.eval()

Epoch 1: val_logloss=0.6811, val_acc=0.7674
Epoch 2: val_logloss=0.2706, val_acc=0.9435
Epoch 3: val_logloss=0.1005, val_acc=0.9767
Epoch 4: val_logloss=0.0659, val_acc=0.9834
Epoch 5: val_logloss=0.0496, val_acc=0.9900
Epoch 6: val_logloss=0.0435, val_acc=0.9900
Epoch 7: val_logloss=0.0462, val_acc=0.9900
Epoch 8: val_logloss=0.0420, val_acc=0.9900
Epoch 9: val_logloss=0.0415, val_acc=0.9900
Epoch 10: val_logloss=0.0447, val_acc=0.9900
Epoch 11: val_logloss=0.0405, val_acc=0.9900
Epoch 12: val_logloss=0.0424, val_acc=0.9900
Epoch 13: val_logloss=0.0414, val_acc=0.9900
Epoch 14: val_logloss=0.0438, val_acc=0.9900
Epoch 15: val_logloss=0.0439, val_acc=0.9900
Epoch 16: val_logloss=0.0454, val_acc=0.9900
Epoch 17: val_logloss=0.0459, val_acc=0.9900
Epoch 18: val_logloss=0.0458, val_acc=0.9900
Epoch 19: val_logloss=0.0449, val_acc=0.9900
Epoch 20: val_logloss=0.0472, val_acc=0.9900
Epoch 21: val_logloss=0.0487, val_acc=0.9900
Early stopping at epoch 21


SimpleNN(
  (net): Sequential(
    (0): Linear(in_features=169, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=64, out_features=5, bias=True)
  )
)

In [334]:
# Process the test set
_, X_test_imputed = impute_missing_genotypes(X_train, X_test)
_, X_test_enc, _ = encode_genotypes(X_train, X_test_imputed)
X_test_scaled = scaler.transform(X_test_enc)
y_test_enc = le.transform(y_test)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

In [335]:
# Evaluate on test set
with torch.no_grad():
    test_out = model(X_test_tensor)
    test_probs = torch.softmax(test_out, dim=1).cpu().numpy()
    test_pred = np.argmax(test_probs, axis=1)

test_logloss = log_loss(y_test_enc, test_probs)
test_acc = accuracy_score(y_test_enc, test_pred)

print(f"Neural Network Test Log Loss: {test_logloss:.4f}")
print(f"Neural Network Test Accuracy: {test_acc:.4f}")

test_logloss_nn = test_logloss
test_acc_nn = test_acc

Neural Network Test Log Loss: 0.0659
Neural Network Test Accuracy: 0.9814


## 7. Neural Network Ensemble

- Ensemble of **5 independently trained neural networks** with different random seeds  
- Predictions averaged (soft voting) to reduce variance  
- Metrics: log loss and accuracy on test set


In [336]:
n_ensemble = 5
ensemble_probs = np.zeros((X_test_scaled.shape[0], output_dim))

for i in range(n_ensemble):
    print(f"Training model {i+1}/{n_ensemble}")
    
    # Re-initialize model
    model = SimpleNN(input_dim, hidden_dim, output_dim)
    
    # Set different random seed
    torch.manual_seed(i)
    np.random.seed(i)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=l2_lambda)
    early_stopper = EarlyStopping(patience=10)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            # L1 regularization
            l1_norm = sum(p.abs().sum() for p in model.parameters())
            loss += l1_lambda * l1_norm
            loss.backward()
            optimizer.step()
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_out = model(X_val_tensor)
            val_probs = torch.softmax(val_out, dim=1).cpu().numpy()
            val_loss = log_loss(y_val_enc, val_probs)
        early_stopper.step(val_loss, model)
        if early_stopper.stop:
            break
    
    # Load best model for this run
    model.load_state_dict(early_stopper.best_model_state)
    model.eval()
    
    # Add predictions to ensemble
    with torch.no_grad():
        out = model(X_test_tensor)
        probs = torch.softmax(out, dim=1).cpu().numpy()
        ensemble_probs += probs

# Average predictions
ensemble_probs /= n_ensemble
ensemble_pred = np.argmax(ensemble_probs, axis=1)

# Metrics on validation set
test_logloss = log_loss(y_test_enc, ensemble_probs)
test_acc = accuracy_score(y_test_enc, ensemble_pred)
print(f"Ensemble Test Log Loss: {test_logloss:.4f}")
print(f"Ensemble Test Accuracy: {test_acc:.4f}")

test_logloss_ens = test_logloss
test_acc_ens = test_acc

Training model 1/5
Training model 2/5
Training model 3/5
Training model 4/5
Training model 5/5
Ensemble Test Log Loss: 0.0657
Ensemble Test Accuracy: 0.9867


/Users/victorloegler/miniconda3/envs/DL2/lib/python3.12/site-packages/sklearn/metrics/_classification.py:259: UserWarning: The y_prob values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


## 8. Evaluation Metrics

- **Log Loss:** measures probability calibration; lower is better  
- **Accuracy:** fraction of correctly predicted labels  

All models evaluated on the **same test set**.

## 9. Results Table

In [337]:
model_names = ["KNN", "Random Forest", "Neural Network", "NN Ensemble"]
test_logloss = [test_logloss_knn, test_logloss_rf, test_logloss_nn, test_logloss_ens]
test_accuracy = [test_acc_knn, test_acc_rf, test_acc_nn, test_acc_ens]

In [338]:
import pandas as pd
from IPython.display import display

# Create DataFrame
results_df = pd.DataFrame({
    "Model": model_names,
    "Test Log Loss": test_logloss,
    "Test Accuracy": [f"{acc*100:.2f}%" for acc in test_accuracy]
})

# Display as a table
display(results_df)

,Model,Test Log Loss,Test Accuracy
0,KNN,0.736862,92.02%
1,Random Forest,0.149283,96.01%
2,Neural Network,0.065885,98.14%
3,NN Ensemble,0.065732,98.67%


**Observation:**  
- Neural network ensemble generally achieves **lowest log loss** and **highest accuracy**.  
- Random Forest is competitive and interpretable.  
- KNN performs reasonably but is less robust for high-dimensional SNPs.

## 10. Conclusion

- Preprocessing (imputation, numeric encoding, scaling) is essential for SNP-based ancestry prediction.  
- Validation set ensures **hyperparameter tuning** and prevents overfitting.  
- Neural networks with **Elastic Net and early stopping** perform well.  
- **Ensembling multiple neural networks** improves generalization and reduces variance.  
- Random Forest is a strong alternative for interpretability and speed.  
- KNN is simple but less effective for high-dimensional genotype data.

**Next steps:**  

- Explore feature selection using SNP importance from Random Forest  
- Combine predictions from KNN, Random Forest, and NN into a hybrid ensemble  
- Experiment with more complex neural network architectures